# Homework 3 - MongoDB

In [1]:
import json
import os
import pymongo

## Read

In [2]:
dataset_dir = './dataset/'
dati_covid_path = os.path.join(dataset_dir, 'dpc-covid19-ita-regioni.json')
dati_vaccini_path = os.path.join(dataset_dir, 'somministrazioni-vaccini-latest.json')

dati_covid = []
with open(dati_covid_path, 'r') as j:
    dati_covid = json.load(j)

dati_vaccini = []
with open(dati_vaccini_path, 'r') as j:
    dati_vaccini = json.load(j)
dati_vaccini = dati_vaccini['data']

In [3]:
client = pymongo.MongoClient('localhost', 27017)

db = client['homework3']

collection_covid = db.dati_covid
collection_vaccini = db.dati_vaccini

print('Available database: ', client.list_database_names())
print('Available collections: ', db.list_collection_names())

Available database:  ['admin', 'config', 'local']
Available collections:  []


In [4]:
# collection_covid.insert_many(dati_covid)
# collection_vaccini.insert_many(dati_vaccini)

In [5]:
print('Available database: ', client.list_database_names())
print('Available collections: ', db.list_collection_names())

Available database:  ['admin', 'config', 'homework3', 'local']
Available collections:  ['dati_vaccini', 'dati_covid']
